In [2]:
# Get the samples.
import json
datas = []
origin_file_list = ["./origin_data/train.json", "./origin_data/valid.json", "./origin_data/test.json"]
for origin_file in origin_file_list:
    with open(origin_file, "r", encoding="utf-8") as f:
        for index, line in enumerate(f):
            line = json.loads(line.strip())
            datas.append(line)
    print("Samples num: {}".format(len(datas)))
print("Examples: ", datas[0:3])

Samples num: 56195
Samples num: 61194
Samples num: 66194
Examples:  [{'sentText': 'Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'relationMentions': [{'em1Text': 'Annandale-on-Hudson', 'em2Text': 'College', 'label': '/location/location/contains'}]}, {'sentText': 'North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'relationMentions': [{'em1Text': 'Carolina', 'em2Text': 'Greensboro', 'label': '/location/location/contains'}]}, {'sentText': "It will be the final movie credited to Debra Hill , a film producer and native of Haddonfield , who produced '' Halloween '' and was considered a pioneering woman in film .", 'relationMentions': [{'em1Text': 'Hill', 'em2Text': 'Haddonfield', 'label': '/people/person/place_of_birth'}]}]


In [3]:
# Get the relations.
relation_types_list = set()
for data in datas:
    for relation in data["relationMentions"]:
        relation_types_list.add(relation["label"])
relation_types_list = list(relation_types_list)
print("relation_types_list: ", relation_types_list)
print("length: ", len(relation_types_list))

relation_types_list:  ['/business/company/place_founded', '/people/deceased_person/place_of_death', '/location/neighborhood/neighborhood_of', '/location/administrative_division/country', '/location/location/contains', '/business/company/founders', '/people/person/children', '/people/person/profession', '/people/ethnicity/geographic_distribution', '/sports/sports_team/location', '/people/person/place_of_birth', '/location/country/administrative_divisions', '/business/company/industry', '/business/company/major_shareholders', '/people/ethnicity/people', '/location/country/capital', '/sports/sports_team_location/teams', '/business/company/advisors', '/people/person/nationality', '/people/person/religion', '/people/person/ethnicity', '/business/person/company', '/business/company_shareholder/major_shareholder_of', '/people/person/place_lived']
length:  24


In [4]:
relation_dict = {
    "business": [],
    "people": [],
    "sports": [],
    "location": []
}
relation_dict_len = {
    "business": 0,
    "people": 0,
    "sports": 0,
    "location": 0
}
for relation in relation_types_list:
    relation_dict[relation.split("/")[1]].append(relation)
    relation_dict_len[relation.split("/")[1]] += 1
print(relation_dict)
print(relation_dict_len)

{'business': ['/business/company/place_founded', '/business/company/founders', '/business/company/industry', '/business/company/major_shareholders', '/business/company/advisors', '/business/person/company', '/business/company_shareholder/major_shareholder_of'], 'people': ['/people/deceased_person/place_of_death', '/people/person/children', '/people/person/profession', '/people/ethnicity/geographic_distribution', '/people/person/place_of_birth', '/people/ethnicity/people', '/people/person/nationality', '/people/person/religion', '/people/person/ethnicity', '/people/person/place_lived'], 'sports': ['/sports/sports_team/location', '/sports/sports_team_location/teams'], 'location': ['/location/neighborhood/neighborhood_of', '/location/administrative_division/country', '/location/location/contains', '/location/country/administrative_divisions', '/location/country/capital']}
{'business': 7, 'people': 10, 'sports': 2, 'location': 5}


### Splite the relation types

In [5]:
# for Intra
intra_train_types_list = relation_dict["location"] + relation_dict["sports"]
intra_dev_types_list = relation_dict["business"]
intra_test_types_list = relation_dict["people"]
print("intra_train: {}, len: {}".format(intra_train_types_list, len(intra_train_types_list)))
print("intra_dev: {}, len: {}".format(intra_dev_types_list, len(intra_dev_types_list)))
print("intra_test: {}, len: {}".format(intra_test_types_list, len(intra_test_types_list)))

intra_train: ['/location/neighborhood/neighborhood_of', '/location/administrative_division/country', '/location/location/contains', '/location/country/administrative_divisions', '/location/country/capital', '/sports/sports_team/location', '/sports/sports_team_location/teams'], len: 7
intra_dev: ['/business/company/place_founded', '/business/company/founders', '/business/company/industry', '/business/company/major_shareholders', '/business/company/advisors', '/business/person/company', '/business/company_shareholder/major_shareholder_of'], len: 7
intra_test: ['/people/deceased_person/place_of_death', '/people/person/children', '/people/person/profession', '/people/ethnicity/geographic_distribution', '/people/person/place_of_birth', '/people/ethnicity/people', '/people/person/nationality', '/people/person/religion', '/people/person/ethnicity', '/people/person/place_lived'], len: 10


In [6]:
# for Inter
import random
random.seed(7)
order = [i for i in range(24)]
random.shuffle(order)
inter_train_types_list = relation_types_list[0:14]
inter_dev_types_list = relation_types_list[14:19]
inter_test_types_list = relation_types_list[19:24]
print("inter_train: {}, len: {}".format(inter_train_types_list, len(inter_train_types_list)))
print("inter_dev: {}, len: {}".format(inter_dev_types_list, len(inter_dev_types_list)))
print("inter_test: {}, len: {}".format(inter_test_types_list, len(inter_test_types_list)))

inter_train: ['/business/company/place_founded', '/people/deceased_person/place_of_death', '/location/neighborhood/neighborhood_of', '/location/administrative_division/country', '/location/location/contains', '/business/company/founders', '/people/person/children', '/people/person/profession', '/people/ethnicity/geographic_distribution', '/sports/sports_team/location', '/people/person/place_of_birth', '/location/country/administrative_divisions', '/business/company/industry', '/business/company/major_shareholders'], len: 14
inter_dev: ['/people/ethnicity/people', '/location/country/capital', '/sports/sports_team_location/teams', '/business/company/advisors', '/people/person/nationality'], len: 5
inter_test: ['/people/person/religion', '/people/person/ethnicity', '/business/person/company', '/business/company_shareholder/major_shareholder_of', '/people/person/place_lived'], len: 5


### Splite the samples based on the relation types list

In [7]:
# Intra data
import copy
intra_types_list = [intra_train_types_list, intra_dev_types_list, intra_test_types_list]
intra_datas = [[] for i in range(3)] # Train, Dev, Test
for data in datas:
    for i in range(3):
        for relation in data["relationMentions"]:
            if relation["label"] in intra_types_list[i]:
                intra_datas[i].append(copy.copy(data))
                break
print(intra_datas[0][0:2])
for i in range(3):
    for intra_data in intra_datas[i]:
        intraRelationMentions = []
        for relation in intra_data["relationMentions"]:
            if relation["label"] in intra_types_list[i]:
                intraRelationMentions.append(relation)
        assert len(intraRelationMentions) != 0
        intra_data["relationMentions"] = intraRelationMentions
print(intra_datas[0][0:2])
print("Train: {}, Dev: {}, Test: {}.".format(len(intra_datas[0]), len(intra_datas[1]), len(intra_datas[2])))

[{'sentText': 'Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'relationMentions': [{'em1Text': 'Annandale-on-Hudson', 'em2Text': 'College', 'label': '/location/location/contains'}]}, {'sentText': 'North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'relationMentions': [{'em1Text': 'Carolina', 'em2Text': 'Greensboro', 'label': '/location/location/contains'}]}]
[{'sentText': 'Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'relationMentions': [{'em1Text': 'Annandale-on-Hudson', 'em2Text': 'College', 'label': '/location/location/contains'}]}, {'sentText': 'North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'relationMentions': [{'em1Text': 'Carolina', 'em2Text': 'Greensboro', 'label': '/location/location/contains'}]}]
Train: 45451, Dev: 6082, Test: 16711.


In [9]:
# Save intra data
import os
import json
if not os.path.exists("intra_data"):
    os.mkdir("intra_data")
intra_file_names = ["./intra_data/train.json", "./intra_data/dev.json", "./intra_data/test.json"]
for index, file_name in enumerate(intra_file_names):
    with open(file_name, "w", encoding="utf-8") as f:
        for intra_data in intra_datas[index]:
            write_str = json.dumps(intra_data) + "\n"
            f.write(write_str)

In [10]:
# Inter data
import copy
inter_types_list = [inter_train_types_list, inter_dev_types_list, inter_test_types_list]
inter_datas = [[] for i in range(3)] # Train, Dev, Test
for data in datas:
    for i in range(3):
        for relation in data["relationMentions"]:
            if relation["label"] in inter_types_list[i]:
                inter_datas[i].append(copy.copy(data))
                break
print(inter_datas[0][0:2])
for i in range(3):
    for inter_data in inter_datas[i]:
        interRelationMentions = []
        for relation in inter_data["relationMentions"]:
            if relation["label"] in inter_types_list[i]:
                interRelationMentions.append(relation)
        assert len(interRelationMentions) != 0
        inter_data["relationMentions"] = interRelationMentions
print(inter_datas[0][0:2])
print("Train: {}, Dev: {}, Test: {}.".format(len(inter_datas[0]), len(inter_datas[1]), len(inter_datas[2])))

[{'sentText': 'Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'relationMentions': [{'em1Text': 'Annandale-on-Hudson', 'em2Text': 'College', 'label': '/location/location/contains'}]}, {'sentText': 'North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'relationMentions': [{'em1Text': 'Carolina', 'em2Text': 'Greensboro', 'label': '/location/location/contains'}]}]
[{'sentText': 'Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'relationMentions': [{'em1Text': 'Annandale-on-Hudson', 'em2Text': 'College', 'label': '/location/location/contains'}]}, {'sentText': 'North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'relationMentions': [{'em1Text': 'Carolina', 'em2Text': 'Greensboro', 'label': '/location/location/contains'}]}]
Train: 50543, Dev: 14362, Test: 12284.


In [11]:
# Save inter data
import os
import json
if not os.path.exists("inter_data"):
    os.mkdir("inter_data")
inter_file_names = ["./inter_data/train.json", "./inter_data/dev.json", "./inter_data/test.json"]
for index, file_name in enumerate(inter_file_names):
    with open(file_name, "w", encoding="utf-8") as f:
        for inter_data in inter_datas[index]:
            write_str = json.dumps(inter_data) + "\n"
            f.write(write_str)